# Group 2: Dekker, Hauser, Tassone, Vogel
## buy/hodl/sell

In [3]:
##### packages
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
import pandas as pd


In [4]:
##### reading data
#AH
df14 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2014_Financial_Data.csv')
df15 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2015_Financial_Data.csv')
df16 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2016_Financial_Data.csv')
df17 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2017_Financial_Data.csv')
df18 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\2018_Financial_Data.csv')


df_sp500 = pd.read_csv(r'C:\Users\Aaron\Documents\Studium UZH\Bachelor\6. Semester\_Machine Learning\project\raw data\sp-500-historical-annual-returns.csv')

In [5]:
df_sp500 = df_sp500[86:91]

df14['year'] = 2014
df15['year'] = 2015
df16['year'] = 2016
df17['year'] = 2017
df18['year'] = 2018

df14['recommendation'] = 0
df15['recommendation'] = 0
df16['recommendation'] = 0
df17['recommendation'] = 0
df18['recommendation'] = 0

df14.rename(columns={"2015 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df15.rename(columns={"2016 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df16.rename(columns={"2017 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df17.rename(columns={"2018 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df18.rename(columns={"2019 PRICE VAR [%]": "PRICE_VAR"},inplace=True)

df_all = pd.concat([df14, df15, df16, df17, df18], ignore_index=True)

In [31]:
##### some minor EDA
df14.columns
df14.info()
df14['Class']
df_all.describe()

##### check missing values
df14.isnull().sum().sum()
df15.isnull().sum().sum()
df16.isnull().sum().sum()
df17.isnull().sum().sum()
df18.isnull().sum().sum()
df_all.isnull().sum().sum()

[3807,
 3807,
 1956,
 1596,
 1596,
 1585,
 1547,
 1547,
 1536,
 1465,
 1424,
 1296,
 1296,
 1277,
 1274,
 1243,
 1243,
 1243,
 1118,
 1118,
 1064,
 1053,
 1049,
 1037,
 1037,
 1031,
 1016]

RangeIndex(start=0, stop=3808, step=1)

RangeIndex(start=0, stop=3808, step=1)

RangeIndex(start=0, stop=3808, step=1)

RangeIndex(start=0, stop=3808, step=1)

...

RangeIndex(start=0, stop=3808, step=1)

RangeIndex(start=0, stop=3808, step=1)

RangeIndex(start=0, stop=3808, step=1)

RangeIndex(start=0, stop=3808, step=1)

[]

In [42]:
#function to give back the colnames of cols with more than 1000 NAs (and therefore possibly to retire from the df)
def name_gettr(data):
    res = {}
    for el in data[:1]:
        if data[el].isnull().sum() > 1000:
            res[el] = data[el].isnull().sum()
    return res
name_gettr(df14)



#df14.drop(['operatingCycle', 'cashConversionCycle', 'shortTermCoverageRatios', 'cashFlowToDebtRatio', 'shortTermCoverageRatios', 'cashFlowCoverageRatios'], axis=1).isnull().sum().idxmax() 
#df14['cashFlowCoverageRatios'].isnull().sum()
#var=[x for x in df14.isnull().sum() if x > 1000]
#var.sort(reverse=True)
#var

#var2 = []
#for i in df14.isnull().sum():
#    if i < 1000:
#        continue
#    elif i in var:
#        #var2.append(df14)
#        df14.index        
#var2
#df14.isnull().sum()


# operatingCycle (3807 NAs), cashConversionCycle (3807 NAs), shortTermCoverageRatios (1956 NAs), 'cashFlowCoverageRatios' (1596),

{'Net Debt': 1277,
 'Other Assets': 1274,
 'priceCashFlowRatio': 1053,
 'priceEarningsToGrowthRatio': 1296,
 'ebtperEBIT': 1118,
 'niperEBT': 1243,
 'effectiveTaxRate': 1243,
 'nIperEBT': 1243,
 'eBTperEBIT': 1118,
 'operatingCycle': 3807,
 'cashConversionCycle': 3807,
 'cashFlowToDebtRatio': 1596,
 'freeCashFlowOperatingCashFlowRatio': 1016,
 'cashFlowCoverageRatios': 1596,
 'shortTermCoverageRatios': 1956,
 'dividendPayoutRatio': 1296,
 'Net Debt to EBITDA': 1424,
 'Weighted Average Shares Diluted Growth': 1049,
 '10Y Revenue Growth (per Share)': 1547,
 '5Y Revenue Growth (per Share)': 1037,
 '10Y Operating CF Growth (per Share)': 1536,
 '5Y Operating CF Growth (per Share)': 1031,
 '10Y Net Income Growth (per Share)': 1547,
 '5Y Net Income Growth (per Share)': 1037,
 '10Y Shareholders Equity Growth (per Share)': 1585,
 '5Y Shareholders Equity Growth (per Share)': 1064,
 '10Y Dividend per Share Growth (per Share)': 1465}

In [47]:
df14['Revenue'].isnull().sum()

44

## Topics to consider
* imputing (e.g. KNN, https://www.kaggle.com/priyankasachdeva20/catboost-model-to-classify-buy-or-sell-stocks/comments line 11)
* 0s in df
* outliers
* feature engineering
* class imbalance
* analysis